In [3]:
import json
import pandas as pd
from pandas import json_normalize


# Read the JSON file
with open('data/restaurants.json', 'r') as f:
    data = json.load(f)

# Create a DataFrame with expanded location fields
df = json_normalize(data, record_path='locations',meta=['name', 'cuisine'],)

df.groupby("cuisine").agg("count").reset_index()

,cuisine,address,area,phoneNumber,price,rating,reviews,coordinate.lat,coordinate.lng,website,name
0,American,21,20,20,21,21,21,21,21,14,21
1,Asian,74,74,72,74,74,74,74,74,43,74
2,Bakeries & Cafes,37,37,36,37,37,37,37,37,28,37
3,Bar & Grill,44,44,44,44,44,44,44,44,34,44
4,Barbecue,14,14,14,14,14,14,14,14,14,14
5,Breakfast,10,10,9,10,10,10,10,10,6,10
6,Dessert,16,16,16,16,16,16,16,16,12,16
7,Fast Food,14,14,13,14,14,14,14,14,11,14
8,Fine Dining,6,6,6,6,6,6,6,6,6,6
9,International,14,14,13,14,14,14,14,14,8,14


In [104]:
import random
import math
import numpy as np
from geopy import distance

class Coordinate:
    def __init__(self, lat, long):
        self.latitude = lat
        self.longitude = long

def random_restaurant(df: pd.DataFrame) -> pd.DataFrame:
    index = random.randint(1, len(df))
    return df.iloc[index]

def filter_cuisine(cuisine: str, df: pd.DataFrame) -> pd.DataFrame:
    return df[df["cuisine"] == cuisine]

def filter_range(current_location: Coordinate, miles: float, df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    df = df.dropna(subset=["coordinate.lat", "coordinate.lng"])
    df["distance"] = df.apply(
        lambda row: distance.distance(
            (current_location.latitude, current_location.longitude),
            (row["coordinate.lat"], row["coordinate.lng"])
        ).miles,
        axis=1
    )
    return df[df["distance"] <= miles].sort_values(by="distance")

In [105]:
filter_range(Coordinate(40.11041, -88.22779), 1, df)

,address,area,phoneNumber,price,rating,reviews,coordinate.lat,coordinate.lng,website,name,cuisine,distance
220,"1401 W Green St, Champaign, Illinois",Campustown,2172394737,1,3.9,13.0,40.109392,-88.227219,eatgarbanzo.com,Garbanzo Mediterranean Fresh,Fast Food,0.076455
88,"1401 W Green St., Urbana, Illinois",Campustown,2173281886,16,4.2,92.0,40.109392,-88.227219,NaN,Sushi San,Asian,0.076455
253,"629 E Green St., Champaign, Illinois",Campustown,2172983198,1,4.4,434.0,40.110196,-88.229232,miazas.com,Mia Za's,Italian,0.077824
81,"616 E Green St, Champaign, Illinois",Campustown,2173443483,2,4.3,128.0,40.110427,-88.229415,NaN,Spoon House Korean Kitchen,Asian,0.086110
85,"619 S Wright St., Champaign, Illinois",Campustown,2179540493,1,3.9,212.0,40.111337,-88.229000,sushiichibanchampaign.com,Sushi Ichiban,Asian,0.090566
...,...,...,...,...,...,...,...,...,...,...,...,...
1,"1 E Main St., Champaign, Illinois",Downtown Champaign,2172393505,2,4.1,474.0,40.118161,-88.243282,biggrovetavern.com,Big Grove Tavern,American,0.979522
154,"2 E Main St, Champaign, Illinois",Downtown Champaign,2173593148,2,4.2,558.0,40.118028,-88.243408,guidosbar.com,Guido's Bur & Grill,Bar & Grill,0.980170
98,"208 W Main Street, Urbana, Illinois",Downtown Urbana,NaN,11,4.5,54.0,40.112581,-88.209395,cafeteriaandcompany.com,Cafeteria & Company,Bakeries & Cafes,0.985913
2,"111 N Race St, Urbana, Illinois",Downtown Urbana,2173281811,2,4.5,1069.0,40.112956,-88.209375,couriersilvercreek.com,Courier Cafe,American,0.991214
